# Exploring CMS Nursing Home Data

In this notebook we explore the Nursing Home Provider Info dataset [available from CMS](https://data.cms.gov/provider-data/dataset/4pq5-n9py) and test its viability for use in our model.

In [ ]:
import pandas as pd
import numpy as np

import requests

## Calling the CMS API

This works quite well for pulling NC data only. The API has a 500 row limit per call, but this is not an issue since the total number of nursing homes in NC is less than 500.

The format of the JSON payload is a list of dicts, with field names for each row. Therefore we use `from_dict` rather than `read_json` to read the JSON file into a dataframe.

In [ ]:
offset = 0

In [ ]:
def call_cms_api(
    state: str,
    offset: int,
    url: str = "https://data.cms.gov/provider-data/api/1/datastore/sql",
    database_id: str = "ef61ab97-218e-57b4-bc2c-d01551e4a614",
) -> pd.DataFrame:
    headers = {
        "accept": "application/json",
    }
    params = (
        (
            "query",
            f'[SELECT * FROM {database_id}][WHERE provider_state == "{state}"][LIMIT 500 OFFSET {offset}];',
        ),
        ("show_db_columns", "true"),
    )
    response = requests.get(url=url, headers=headers, params=params)
    json = response.json()
    return pd.DataFrame.from_dict(json)

In [ ]:
dfs = []

In [ ]:
dfs.append(call_cms_api(state="CA", offset=offset))

In [ ]:
while len(dfs[-1]) == 500:
    offset += 500
    dfs.append(call_cms_api(state="CA", offset=offset))

In [ ]:
cms = pd.concat(dfs)

## CMS dataset contents

Number of certified beds is available for all nursing homes. Average number of residents per day is missing for 1.4% of nursing homes. All of these include the value "10" in the column `average_number_of_residents_per_day_footnote`, but I can't figure out exactly what that means. The data dictionary was no help.

There are lots of other variables in this dataset that we could consider using. The staff per occupant figure could be particularly helpful if we want to model staff interactions with patients.

In [ ]:
for col in cms.columns:
    print(col)

# NOTE: add `"notebook.output.textLineLimit": 500` to .vscode/settings.json to print the full output

In [ ]:
cms.number_of_certified_beds = cms.number_of_certified_beds.astype(int)

cms.number_of_certified_beds.describe()

In [ ]:
missingness = round(cms.number_of_certified_beds.isna().sum() / len(cms.number_of_certified_beds) * 100, 2)
print(f'Missingness = {missingness}%')

In [ ]:
cms.average_number_of_residents_per_day = np.where(
    cms.average_number_of_residents_per_day.eq(''),
    None,
    cms.average_number_of_residents_per_day,
)
cms.average_number_of_residents_per_day = cms.average_number_of_residents_per_day.astype(float)

cms.average_number_of_residents_per_day.describe()


In [ ]:
missingness = round(cms.average_number_of_residents_per_day.isna().sum() / len(cms.average_number_of_residents_per_day) * 100, 2)
print(f'Missingness = {missingness}%')

## Merging with nursing home locations

Locations of nursing homes for the location model are in `data/locations/nursing_homes.csv`. We need to combine locations with the occupancy data in this file.

The Facility ID in the nursing homes data does not match the Federal Provider Number in the CMS data, so we'll need to find another way.


In [ ]:
nh = pd.read_csv('data/locations/nursing_homes.csv')

### Option 1 -  string matching

Pretty ugly. We'd have to go through and manually correct as we did with hospitals. Not the end of the world but not fun.

In [ ]:
from string import punctuation

In [ ]:
punct = punctuation + "’‘"

In [ ]:
substrings_to_remove = ['INC', "LLC"]

In [ ]:
def clean_string(column: pd.Series, chars_to_remove: str, substrings_to_remove: list):
    col = column.copy()
    col = col.str.upper()
    for c in chars_to_remove:
        col = col.str.replace(c, "", regex=False)
    for s in substrings_to_remove:
        col = col.str.replace(s, "", regex=False)
    col = col.str.strip()
    return col


In [ ]:
cms['name_clean'] = clean_string(cms.provider_name, punct, substrings_to_remove)

In [ ]:
nh['name_clean'] = clean_string(nh.Name, punct, substrings_to_remove)

In [ ]:
merged = cms.merge(nh, how='outer', on='name_clean', indicator=True)

In [ ]:
merged._merge.value_counts()

### Option 2 - regenerate lat and lon using addresses from CMS data

Geocoding from CMS addresses is accurate when it works, but doesn't have a very good success rate.

In [ ]:
# Installing geocoder directly here rather than adding it to requirements.txt, 
# since we're still at the experimental stage and not sure we'll end up using this.

import sys
!{sys.executable} -m pip install geocoder

In [ ]:
import geocoder
from matplotlib import pyplot as plt

In [ ]:
matches = merged.query("_merge == 'both'").copy()

In [ ]:
geo = [geocoder.osm(address).json for address in matches.location]

In [ ]:
success_rate = round(sum(g is not None for g in geo) / len(geo) * 100, 2)
print(f'Geocoding successful for {success_rate}% of addresses')

In [ ]:
matches['LAT_CMS'] = [g['lat'] if g else None for g in geo]
matches['LON_CMS'] = [g['lng'] if g else None for g in geo]

In [ ]:
plt.scatter(matches['LAT'], matches['LAT_CMS'])

In [ ]:
plt.scatter(matches['LON'], matches['LON_CMS'])